## --- Day 9: Rope Bridge ---
```
This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

..##..
...##.
.####.
....#.
s###..

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?
```

In [1]:
from typing import List, Tuple


class Rope:
    directions_mapping = {"R": +1, "L": -1, "U": +1, "D": -1}

    def __init__(self, H_state: Tuple[int] = (0, 0), T_state: Tuple[int] = (0, 0)):
        self.H_state = H_state
        self.T_state = T_state
        self.H_positions_history: List[Tuple[int]] = [H_state]
        self.T_positions_history: List[Tuple[int]] = [H_state]

    def move(self, move_command: str):
        direction, number_of_steps = self.parse_move_command(move_command)
        # start walking
        for n in range(number_of_steps):
            if direction in ["R", "L"]:
                # move head
                H_x = self.H_state[0] + Rope.directions_mapping[direction]
                self.H_state = (H_x, self.H_state[1])
                # move tail
                if abs(self.T_state[0] - self.H_state[0]) > 1:
                    if self.T_state[1] == self.H_state[1]:
                        T_x = self.T_state[0] + Rope.directions_mapping[direction]
                        self.T_state = (T_x, self.T_state[1])
                    else:
                        # need to make diagonal move
                        T_x = self.T_state[0] + Rope.directions_mapping[direction]
                        T_y = self.H_state[1]
                        self.T_state = (T_x, T_y)

            else:
                # move head
                H_y = self.H_state[1] + Rope.directions_mapping[direction]
                self.H_state = (self.H_state[0], H_y)
                # move tail
                if abs(self.T_state[1] - self.H_state[1]) > 1:
                    if self.T_state[0] == self.H_state[0]:
                        T_y = self.T_state[1] + Rope.directions_mapping[direction]
                        self.T_state = (self.T_state[0], T_y)
                    else:
                        # need to make diagonal move
                        T_y = self.T_state[1] + Rope.directions_mapping[direction]
                        T_x = self.H_state[0]
                        self.T_state = (T_x, T_y)

            self.H_positions_history.append(self.H_state)
            self.T_positions_history.append(self.T_state)

    @staticmethod
    def parse_move_command(move_command: str) -> Tuple[str, int]:
        direction, number_of_steps = move_command.split(" ")
        assert direction in Rope.directions_mapping
        return direction, int(number_of_steps)

In [2]:
rope = Rope()
assert rope.parse_move_command("R 1") == ("R", 1)

test_moves = ["R 4", "U 4", "L 3", "D 1", "R 4", "D 1", "L 5", "R 2"]
for test_move in test_moves:
    rope.move(test_move)

assert rope.H_positions_history == [
    (0, 0),
    (1, 0),
    (2, 0),
    (3, 0),
    (4, 0),
    (4, 1),
    (4, 2),
    (4, 3),
    (4, 4),
    (3, 4),
    (2, 4),
    (1, 4),
    (1, 3),
    (2, 3),
    (3, 3),
    (4, 3),
    (5, 3),
    (5, 2),
    (4, 2),
    (3, 2),
    (2, 2),
    (1, 2),
    (0, 2),
    (1, 2),
    (2, 2),
]

assert rope.T_positions_history == [
    (0, 0),
    (0, 0),
    (1, 0),
    (2, 0),
    (3, 0),
    (3, 0),
    (4, 1),
    (4, 2),
    (4, 3),
    (4, 3),
    (3, 4),
    (2, 4),
    (2, 4),
    (2, 4),
    (2, 4),
    (3, 3),
    (4, 3),
    (4, 3),
    (4, 3),
    (4, 3),
    (3, 2),
    (2, 2),
    (1, 2),
    (1, 2),
    (1, 2),
]

assert len(set(rope.T_positions_history)) == 13

In [3]:
with open("data/day_9.txt", "r") as f:
    moves = f.read().split("\n")

In [4]:
rope = Rope()
for move in moves:
    rope.move(move)

In [5]:
len(set(rope.T_positions_history))

6067

In [6]:
assert len(set(rope.T_positions_history)) == 6067

## --- Part Two ---

```
A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:

== Initial State ==

......
......
......
......
H.....  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)

== R 4 ==

......
......
......
......
1H....  (1 covers 2, 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
21H...  (2 covers 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
321H..  (3 covers 4, 5, 6, 7, 8, 9, s)

......
......
......
......
4321H.  (4 covers 5, 6, 7, 8, 9, s)

== U 4 ==

......
......
......
....H.
4321..  (4 covers 5, 6, 7, 8, 9, s)

......
......
....H.
.4321.
5.....  (5 covers 6, 7, 8, 9, s)

......
....H.
....1.
.432..
5.....  (5 covers 6, 7, 8, 9, s)

....H.
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

== L 3 ==

...H..
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

..H1..
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

.H1...
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

..1...
.H.2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== R 4 ==

..1...
..H2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

..1...
...H..  (H covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...1H.  (1 covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21H
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

......
...21.
..43.H
.5....
6.....  (6 covers 7, 8, 9, s)

== L 5 ==

......
...21.
..43H.
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21.
..4H..  (H covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
..H1..  (H covers 4; 1 covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
.H13..  (1 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
H123..  (2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

== R 2 ==

......
......
.H23..  (H covers 1; 2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
.1H3..  (H covers 2, 4)
.5....
6.....  (6 covers 7, 8, 9, s)

Now, you need to keep track of the positions the new tail, 9, visits. In this example, the tail never moves, and so it only visits 1 position. However, be careful: more types of motion are possible than before, so you might want to visually compare your simulated rope to the one above.


Simulate your complete series of motions on a larger rope with ten knots. How many positions does the tail of the rope visit at least once?
```

In [112]:
from typing import List, Tuple
from copy import deepcopy


class GenericRope:
    """I know how to write it in functional style, but I don't have time to rewrite this 😢."""

    directions_mapping = {"R": +1, "L": -1, "U": +1, "D": -1}

    def __init__(self, initial_state: Tuple[int] = (0, 0), n_knots: int = 10):
        self.states = {i: initial_state for i in range(n_knots)}
        self.knots = list(self.states.keys())
        self.positions_history: List[Tuple[int]] = [deepcopy(self.states)]

    def move_head(self, direction: str):
        if direction in ["R", "L"]:
            x = self.states[0][0] + Rope.directions_mapping[direction]
            self.states[0] = (x, self.states[0][1])
        else:
            y = self.states[0][1] + Rope.directions_mapping[direction]
            self.states[0] = (self.states[0][0], y)

    def move_subsequent_knot(self, direction: str, knot_index: int):
        if direction in ["R", "L"]:
            if abs(self.states[knot_index - 1][0] - self.states[knot_index][0]) > 1:
                if self.states[knot_index - 1][1] == self.states[knot_index][1]:
                    x = self.states[knot_index][0] + Rope.directions_mapping[direction]
                    self.states[knot_index] = (x, self.states[knot_index][1])
                else:
                    # need to make diagonal move
                    x = self.states[knot_index][0] + Rope.directions_mapping[direction]
                    y = self.states[knot_index][1] + Rope.directions_mapping[direction]
                    self.states[knot_index] = (x, y)
        else:
            if abs(self.states[knot_index - 1][1] - self.states[knot_index][1]) > 1:
                if self.states[knot_index - 1][0] == self.states[knot_index][0]:
                    y = self.states[knot_index][1] + Rope.directions_mapping[direction]
                    self.states[knot_index] = (self.states[knot_index][0], y)
                else:
                    # need to make diagonal move
                    # if knot_index == 2:
                    #     import pdb;pdb.set_trace()
                    x = self.states[knot_index][0] + Rope.directions_mapping[direction]
                    y = self.states[knot_index][1] + Rope.directions_mapping[direction]
                    self.states[knot_index] = (x, y)

    def move(self, move_command: str):
        direction, number_of_steps = self.parse_move_command(move_command)
        # start walking
        print("="*100)
        print(move_command)
        print("="*100)
        for n in range(number_of_steps):
            self.move_head(direction)
            for knot_index in self.knots[1:]:
                self.move_subsequent_knot(direction, knot_index)
                print(knot_index, self.states)
            print("$"*100)
            self.positions_history.append(deepcopy(self.states))
            
        print("^"*100)
        print(self.states)
        print("^"*100)


    @staticmethod
    def parse_move_command(move_command: str) -> Tuple[str, int]:
        direction, number_of_steps = move_command.split(" ")
        assert direction in Rope.directions_mapping
        return direction, int(number_of_steps)

In [117]:
def parse_move_command(move_command: str) -> Tuple[str, int]:
    direction, number_of_steps = move_command.split(" ")
    assert direction in Rope.directions_mapping
    return direction, int(number_of_steps)

In [145]:
initial_state: Tuple[int] = (0, 0)
directions_mapping = {"R": +1, "L": -1, "U": +1, "D": -1}
n_knots = 10
states = {i: initial_state for i in range(n_knots)}
knots = list(states.keys())
moves = ["R 4", "U 4"]

for move in moves:
    direction, number_of_steps = parse_move_command(move)
    for n in range(number_of_steps):
        # move head
        if direction in ["R", "L"]:
            x = states[0][0] + directions_mapping[direction]
            states[0] = (x, states[0][1])
        else:
            y = states[0][1] + directions_mapping[direction]
            states[0] = (states[0][0], y)
        # print("*"*100)
        # print(n)
        # print("*"*100)
        # move other knots
        for knot_index in knots[1:]:
            if abs(states[knot_index - 1][1] - states[knot_index][1])>1 or abs(states[knot_index - 1][0] - states[knot_index][0]) > 1:
                if direction in ["R", "L"]:
                # if abs(states[knot_index - 1][0] - states[knot_index][0]) > 1:
                    if states[knot_index - 1][1] == states[knot_index][1]:
                        x = states[knot_index][0] + directions_mapping[direction]
                        states[knot_index] = (x, states[knot_index][1])
                    else:
                        # need to make diagonal move
                        x = states[knot_index][0] + directions_mapping[direction]
                        y = states[knot_index][1] + directions_mapping[direction]
                        states[knot_index] = (x, y)
                else:

                # print(states)
                # if abs(states[knot_index - 1][1] - states[knot_index][1])>1 or abs(states[knot_index - 1][0] - states[knot_index][0]) > 1:
                    if states[knot_index - 1][0] == states[knot_index][0]:
                        y = states[knot_index][1] + Rope.directions_mapping[direction]
                        states[knot_index] = (states[knot_index][0], y)
                    else:
                        # need to make diagonal move
                        x = states[knot_index][0] + directions_mapping[direction]
                        y = states[knot_index][1] + directions_mapping[direction]
                        states[knot_index] = (x, y)
        

In [ ]:
3,1,

```
== U 4 ==

......
......
......
....H.
4321..  (4 covers 5, 6, 7, 8, 9, s)

......
......
....H.
.4321.
5.....  (5 covers 6, 7, 8, 9, s)
```

In [137]:
states

{0: (4, 4),
 1: (4, 3),
 2: (4, 2),
 3: (2, 1),
 4: (0, 0),
 5: (0, 0),
 6: (0, 0),
 7: (0, 0),
 8: (0, 0),
 9: (0, 0)}

In [113]:
generic_rope = GenericRope(n_knots=5)
test_moves = ["R 4", "U 4"] #, "L 3", "D 1", "R 4", "D 1", "L 5", "R 2"]
for test_move in test_moves:
    generic_rope.move(test_move)

R 4
1 {0: (1, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
2 {0: (1, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
3 {0: (1, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
4 {0: (1, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1 {0: (2, 0), 1: (1, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
2 {0: (2, 0), 1: (1, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
3 {0: (2, 0), 1: (1, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
4 {0: (2, 0), 1: (1, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1 {0: (3, 0), 1: (2, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0)}
2 {0: (3, 0), 1: (2, 0), 2: (1, 0), 3: (0, 0), 4: (0, 0)}
3 {0: (3, 0), 1: (2, 0), 2: (1, 0), 3: (0, 0), 4: (0, 0)}
4 {0: (3, 0), 1: (2, 0), 2: (1, 0), 3: (0, 0), 4: (0, 0)}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

In [83]:
# generic_rope.positions_history

In [47]:
generic_rope = GenericRope(n_knots=3)
generic_rope.move("R 4")

In [35]:
generic_rope.move("U 4")

U 4
{0: (4, 1), 1: (3, 0), 2: (2, 0)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}, {0: (1, 0), 1: (0, 0), 2: (0, 0)}, {0: (2, 0), 1: (1, 0), 2: (0, 0)}, {0: (3, 0), 1: (2, 0), 2: (1, 0)}, {0: (4, 0), 1: (3, 0), 2: (2, 0)}]
{0: (4, 2), 1: (3, 1), 2: (2, 0)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}, {0: (1, 0), 1: (0, 0), 2: (0, 0)}, {0: (2, 0), 1: (1, 0), 2: (0, 0)}, {0: (3, 0), 1: (2, 0), 2: (1, 0)}, {0: (4, 0), 1: (3, 0), 2: (2, 0)}, {0: (4, 1), 1: (3, 0), 2: (2, 0)}]
{0: (4, 3), 1: (3, 2), 2: (2, 1)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}, {0: (1, 0), 1: (0, 0), 2: (0, 0)}, {0: (2, 0), 1: (1, 0), 2: (0, 0)}, {0: (3, 0), 1: (2, 0), 2: (1, 0)}, {0: (4, 0), 1: (3, 0), 2: (2, 0)}, {0: (4, 1), 1: (3, 0), 2: (2, 0)}, {0: (4, 2), 1: (3, 1), 2: (2, 0)}]
{0: (4, 4), 1: (3, 3), 2: (2, 2)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}, {0: (1, 0), 1: (0, 0), 2: (0, 0)}, {0: (2, 0), 1: (1, 0), 2: (0, 0)}, {0: (3, 0), 1: (2, 0), 2: (1, 0)}, {0: (4, 0), 1: (3, 0), 2: (2, 0)}, {0: (4, 1), 1: (3, 0), 2: (2, 0)}, {0: (4, 2), 1: (

In [36]:
generic_rope = GenericRope(n_knots=3)

In [37]:
generic_rope.positions_history

[{0: (0, 0), 1: (0, 0), 2: (0, 0)}]

In [38]:
generic_rope.move("R 4")

R 4
{0: (1, 0), 1: (0, 0), 2: (0, 0)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}]
{0: (2, 0), 1: (1, 0), 2: (0, 0)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}, {0: (1, 0), 1: (0, 0), 2: (0, 0)}]
{0: (3, 0), 1: (2, 0), 2: (1, 0)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}, {0: (1, 0), 1: (0, 0), 2: (0, 0)}, {0: (2, 0), 1: (1, 0), 2: (0, 0)}]
{0: (4, 0), 1: (3, 0), 2: (2, 0)}
[{0: (0, 0), 1: (0, 0), 2: (0, 0)}, {0: (1, 0), 1: (0, 0), 2: (0, 0)}, {0: (2, 0), 1: (1, 0), 2: (0, 0)}, {0: (3, 0), 1: (2, 0), 2: (1, 0)}]


In [39]:
generic_rope.positions_history

[{0: (0, 0), 1: (0, 0), 2: (0, 0)},
 {0: (1, 0), 1: (0, 0), 2: (0, 0)},
 {0: (2, 0), 1: (1, 0), 2: (0, 0)},
 {0: (3, 0), 1: (2, 0), 2: (1, 0)},
 {0: (4, 0), 1: (3, 0), 2: (2, 0)}]

In [264]:
generic_rope = GenericRope(n_knots=3)
test_moves = ["R 4", "U 4", "L 3", "D 1", "R 4", "D 1", "L 5", "R 2"]
for test_move in test_moves:
    generic_rope.move(test_move)

R 4
{0: (1, 0), 1: (0, 0), 2: (0, 0)}
{0: (2, 0), 1: (1, 0), 2: (0, 0)}
{0: (3, 0), 1: (2, 0), 2: (1, 0)}
{0: (4, 0), 1: (3, 0), 2: (2, 0)}
U 4
{0: (4, 1), 1: (3, 0), 2: (2, 0)}
{0: (4, 2), 1: (3, 1), 2: (2, 0)}
{0: (4, 3), 1: (3, 2), 2: (2, 1)}
{0: (4, 4), 1: (3, 3), 2: (2, 2)}
L 3
{0: (3, 4), 1: (3, 3), 2: (2, 2)}
{0: (2, 4), 1: (3, 3), 2: (2, 2)}
{0: (1, 4), 1: (2, 3), 2: (2, 2)}
D 1
{0: (1, 3), 1: (2, 3), 2: (2, 2)}
R 4
{0: (2, 3), 1: (2, 3), 2: (2, 2)}
{0: (3, 3), 1: (2, 3), 2: (2, 2)}
{0: (4, 3), 1: (3, 3), 2: (2, 2)}
{0: (5, 3), 1: (4, 3), 2: (3, 2)}
D 1
{0: (5, 2), 1: (4, 3), 2: (3, 2)}
L 5
{0: (4, 2), 1: (4, 3), 2: (3, 2)}
{0: (3, 2), 1: (4, 3), 2: (3, 2)}
{0: (2, 2), 1: (3, 3), 2: (3, 2)}
{0: (1, 2), 1: (2, 3), 2: (3, 2)}
{0: (0, 2), 1: (1, 3), 2: (2, 2)}
R 2
{0: (1, 2), 1: (1, 3), 2: (2, 2)}
{0: (2, 2), 1: (1, 3), 2: (2, 2)}


In [266]:
len(generic_rope.positions_history)

25

In [84]:
rope = GenericRope(n_knots=10)
for move in moves:
    rope.move(move)

In [85]:
len(set([p[9] for p in rope.positions_history]))

349

In [158]:
len(set([p[9] for p in history]))

399